In [1]:
import pandas as pd

from langchain import hub

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper

from langchain.tools import BraveSearch, BaseTool, StructuredTool, tool

from langchain_openai import ChatOpenAI

from langchain.agents import AgentExecutor
from langchain.agents import create_openai_tools_agent

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, chat
from langchain.schema import AIMessage, HumanMessage, SystemMessage

from langchain.pydantic_v1 import BaseModel, Field

from typing import Optional, Type

C:\Users\14102\AppData\Local\Temp\ipykernel_20752\2472464191.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [37]:
# Define some constants 
VERBOSE = True
OPENAI_MODEL = "gpt-4-turbo-preview"
MODEL_TEMP = 0.0
BRAVE_API_KEY='BSAmw298lpAtPuiP5crDrAtOpGPgf5U'

## Compile Electricity Providers

Data from [EIA 2022 report](https://www.eia.gov/electricity/data/eia861/)

In [3]:
# Get list of electricity providers
elec_industry_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Frame_2022.csv")
# Get list of electricity providers
service_territory_df = pd.read_csv("C:\\Users\\14102\\Brown\\Internships\\INL\\Data\\Service_Territory_2022.csv")

service_territory_df = service_territory_df.drop(
    [col for col in service_territory_df.columns if col not in ['Utility Number', 'State', 'County']], axis=1).reset_index(drop=True)

In [4]:
# Only look at investor owned companies for now!
investor_utils = elec_industry_df[elec_industry_df['Ownership'] == 'Investor Owned']
# Drop some columns
investor_utils = investor_utils.drop(
    [col for col in investor_utils.columns if col not in ['Utility Number', 'Utility Name']], axis=1).reset_index(drop=True)

In [5]:
# Join investor owned utils with their reported coverage
investor_util_coverage = investor_utils.merge(service_territory_df, on='Utility Number', how='inner')

## Build Web Searching Tool (Tavily) for Agent

[Tavily link](https://tavily.com/)

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/tavily_search)

In [38]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results = 3)

In [22]:
# response = tavily_tool.invoke({'query': 'Baltimore Gas and Electric climate resilience executive leadership'})

In [99]:
# response#[0]['content']


## Build Backup Search Tool

[Usage in LangChain](https://python.langchain.com/docs/integrations/tools/brave_search)

In [39]:
brave_tool = BraveSearch.from_api_key(api_key=BRAVE_API_KEY, search_kwargs={"count": 4}, description='a search engine. useful to use when other tools do not return results.')

## Get CRMM Text file 

In [97]:
with open('crmm_docs/CRMM_13.txt', "r", encoding='utf-8') as f:
    crmm_raw = f.read()

with open('crmm_docs/CRMM_header.txt', "r", encoding='utf-8') as f:
    crmm_head = f.read()

cap_dict = {}
for num in range(5):
    with open(f'crmm_docs/Capability {num+1}.txt', "r", encoding='utf-8') as f:
        cap = f.read()
    cap_dict[f'cap{num+1}'] = cap

## Build Chat Prompt for Agent

[LangChain Documentation](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [14]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                    template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how infrastructure providers are implementing climate resiliency measures." 
                     "\nYou are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided related to infrastructure development or resilience planning and organization." 
                     " The Climate Resilience Maturity Model is explained below:\n####\n{crmm_info}\n####\n")),
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} is positioned with respect to the Climate Resilience Maturity Model. "
                     "For each capability, provide a one sentence summary, list all the evidence that led to the conclusion, and cite sources. All evidence MUST be directly related to the Climate Resilience Maturity Model. Use as many different relevent sources as possible to gauge their overall position. \n"
                     "**Research Approach**: \n 1. Search for any documents related to {org_name} and climate resilience. \n 2. Search for documents specifically related to capabilities not accounted for in step 1. \n"
                     " Do **not** consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

In [88]:
# OpenAI agents need placeholders for chat history and agent scratchpad.
chat_prompt_cap = ChatPromptTemplate.from_messages(
                # System message gives the model the 'backstory' for the task (eg. how do you act? what info do you have on hand? what are you designed to do?), but does not actually respond to this directly
                [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['crmm_info'], 
                    template="You are an analyst who uses the Climate Resilience Maturity Model to summarize how infrastructure providers are implementing climate resiliency measures." 
                     "\nYou are critical of providers, and only give them credit for progress in a capability if specific and direct evidence is provided related to infrastructure development or resilience planning and organization." 
                     " The Climate Resilience Maturity Model is explained below:\n####\n{crmm_info}\n####\n")),
                chat.MessagesPlaceholder(variable_name='chat_history', optional=True),
                # Human message represents the question actually being asked by a user or service. The model's output is a response to this message.
                chat.HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['org_name', 'cap_info'], template="Summarize how the infrastructure and initiatives of the organization, {org_name} are positioned with respect to the following capability in the Climate Resilience Maturity Model: \n"
                     "####\n{cap_info}\n####\n"
                     "Provide a summary in terms of the capability criteria, and list all the evidence that led to the conclusion, and cite sources. All evidence MUST be related to climate resiliency in some way. Use as many different relevent sources as possible to gauge their overall position. \n"
                     " Do not consider the following topics when collecting evidence:\n -greenhouse gas emissions \n -energy saving and efficiency \n -decarbonization \n -clean energy")),
                chat.MessagesPlaceholder(variable_name='agent_scratchpad')])

## Create the Agent

[LangChain Agent Documentation](https://python.langchain.com/docs/modules/agents/quick_start)

[LangChain Tools Documentation](https://python.langchain.com/docs/modules/agents/tools/)

In [89]:
# Define the model (GPT-4)
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=MODEL_TEMP)

In [90]:
# Define tools available to agent
tools = [brave_tool, tavily_tool]

# Use this function to make agents based on OpenAI models
agent = create_openai_tools_agent(llm, tools, chat_prompt_cap)

In [91]:
# Define agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=VERBOSE)

## Running the Agent Iteratively

In [ ]:
# Define input list: Currently is investor-owned elec utils from MD
md_elec_providers = investor_util_coverage[investor_util_coverage['State'] == 'MD']['Utility Name'].unique()
research_output = {}
# Iterate over every provider in our array
for organization in md_elec_providers:

    ## Compile Agent Input
    input_dict = {
        'crmm_info': crmm_raw,
        'org_name': organization}

    # Run the agent for given org
    agent_output: dict = agent_executor.invoke(input_dict)

    # Record result in dict
    research_output[organization] = agent_output['output']

In [ ]:
rag_output = pd.DataFrame.from_dict(research_output, orient='index', columns=['Research Results']).reset_index(names=['Utility Name'])
rag_output = rag_output.merge(investor_utils, on='Utility Name', how='inner')

## Observing Outputs

In [ ]:
for org, rag_response in research_output.items():
    print(f'RAG Response for {org}:\n')
    print(rag_response)
    print('\n\n\n')

## Custom Search Tool with Backup (WIP)

[LangChain Documentation](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [ ]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions using the internet."
    args_schema: Type[BaseModel] = SearchInput

    def __init__(self, tavily, brave): 
        super(BaseTool, self).__init__()
        self.tavily = tavily
        self.brave = brave

    def _run(
        self, query: str) -> str:
        """Use the tool."""
        output = self.tavily.run(query)
        if not output:
            output = self.brave.run(query)
        
        return output

    async def _arun(
        self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [ ]:
custom_search = CustomSearchTool(tavily_tool, brave_tool)

## Running the Agent (One-Off)

In [100]:
## Running the Agent
test_input_dict = {
    'crmm_info': crmm_head,
    'org_name': 'Maryland Department of Transportation',
    'cap_info': cap_dict['cap1']
}


In [101]:
# Actually runs the RAG! Returns a dict of inputs (from prompt defined above) and output
agent_executor.invoke(test_input_dict)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Maryland Department of Transportation climate resilience lead'}`


[{'url': 'https://www.linkedin.com/in/sandy-hertz-95091715', 'content': 'Director, Office of Climate Change Resilience and Adaptation Maryland Department of Transportation Washington DC-Baltimore Area 845 followers 500+ connections'}, {'url': 'https://www.mdot.maryland.gov/OPCP/MDOT_TRIP_Report_2024_Final.pdf', 'content': 'deploying a variety of climate resilience strategies. Together, the State of Maryland and the Maryland Department of Transportation (MDOT) have sought resilience measures through strategic investments in projects, programs, and at-risk infrastructure. MDOT will continue to work with partners to ensure expanded alignment of resilience measures.'}, {'url': 'https://www.mdot.maryland.gov/OPCP/MDOT_CPRP_2023_Final.pdf', 'content': 'The Maryland Department of Transportation (MDOT) Climate Pollution Reduction Pl

{'crmm_info': 'Climate Resilience Maturity Model (CRMM) for Essential Services Providers\n\n\nExecutive Summary\n\n\nAll US cities are in some stage of awareness and action related to hardening their infrastructure assets and adapting their critical functions and processes to operate reliably in the more challenging conditions wrought by a changing climate. This model offers a mechanism for entities to baseline their present state and set goals for heightened resilience in the near-midterm future. When aggregated across the six sectors assessed, a city’s resilience posture is made visible to all manner of interested stakeholders. Also please note:\n\n\n* Criteria and annual ratings are publicly accessible, though particularly sensitive data may be reserved for USG use.\n* This model is fundamentally multi-sector and is designed to take cross-sector interdependencies into full account (e.g., energy, water, communications, transportation, etc.).\n* Intended to motivate accelerated action

### Delmarva Power's Climate Risk Maturity Model Summary (Issues with mitigation/resilience differentiation)

#### Capability 1: Governance – Executive Level Engagement and Leadership
- **Summary:** Delmarva Power has demonstrated a commitment to addressing climate change and resilience through executive leadership involvement and public statements of intent.
- **Evidence:** The company has made public commitments to address climate change and resilience, including the release of a Responsibility and Impact Report outlining efforts to address climate change. [Source](https://www.delmarva.com/News/Pages/NewsReleases/DelmarvaPowerReleases2020ResponsibilityandImpactReportOutliningtheCompany%E2%80%99sEffortstoHelpDriveEquity,AddressClimateChange.aspx)

#### Capability 2: Climate-Aware Planning
- **Summary:** Delmarva Power is actively engaged in climate-aware planning, with specific programs aimed at energy efficiency and infrastructure improvements.
- **Evidence:** The company has proposed programs expected to save over 80,000-megawatt hours of electricity and has planned more than $430 million worth of improvements through 2024. [Source](https://delawarebusinessnow.com/2023/06/delmarva-power-proposes-residential-energy-efficiency-programs/)

#### Capability 3: Stakeholder and Community Collaboration
- **Summary:** Delmarva Power collaborates with local non-profits and community organizations to support resilience and community needs.
- **Evidence:** The company partners with over 200 local non-profits in Delaware and Maryland to support community resilience and connectedness. [Source](https://www.pr-inside.com/delmarva-power-delivers-more-than-energy-for-its-community-r4982277.htm)

#### Capability 4: Resilience and Adaptation Actions
- **Summary:** Delmarva Power has initiated significant actions to advance clean energy, combat climate change, and build resilience in Maryland.
- **Evidence:** The company announced a major climate change commitment, including a multi-faceted effort to help Maryland achieve its clean energy and climate goals. [Source](https://www.delmarva.com/News/Pages/NewsReleases/DelmarvaPowerTakingMajorActiontoAdvanceCleanEnergy,CombatClimateChangeandBuildResilienceinMaryland.aspx)

#### Capability 5: Customer Engagement and Coordination
- **Summary:** Delmarva Power engages with customers through energy efficiency programs and efforts to enhance electric grid reliability.
- **Evidence:** The company has proposed energy efficiency programs for Delaware customers and has reinforced the local energy grid to reduce electric outages. [Source](https://delawarebusinessnow.com/2023/06/delmarva-power-proposes-residential-energy-efficiency-programs/)

#### Capability 6: Attention to Equity
- **Summary:** Delmarva Power actively engages in supporting economically challenged communities through its planning processes and grant programs.
- **Evidence:** The Sustainable Communities Grant Program provides $75,000 in grants annually for environmental projects and resilience projects in economically challenged communities. [Source](https://delawarebusinesstimes.com/news-briefs/delmarva-open-space-grants/)

Delmarva Power's efforts across these capabilities indicate a proactive approach to climate resilience and adaptation, with a notable emphasis on community collaboration and equity.ased on available evidence.

#### Capability 1: Governance – Executive Level Engagement and Leadership
- **Summary:** Delmarva Power demonstrates initial steps towards climate resilience governance with advocacy for energy assistance but lacks direct evidence of a structured climate resilience governance framework.
- **Evidence:**
  - Advocacy for Low-Income Home Energy Assistance Program (LIHEAP) funding [Source](https://www.marketscreener.com/quote/stock/EXELON-CORPORATION-13963/news/Delmarva-Power-Advocates-for-Continued-Energy-Assistance-Funding-and-Reminds-Customers-To-Act-Now-To-45933752/).

#### Capability 2: Climate-Aware Planning
- **Summary:** Delmarva Power is in the initial stages of climate-aware planning, focusing on energy efficiency and customer savings without specific mention of climate resilience planning.
- **Evidence:**
  - Introduction of energy-saving programs [Source](https://www.delawarepublic.org/delaware-headlines/2023-06-11/delmarva-power-introduces-four-new-energy-saving-programs).

#### Capability 3: Stakeholder and Community Collaboration
- **Summary:** Delmarva Power engages in stakeholder and community collaboration through partnerships and programs aimed at environmental benefits and energy savings, indicating initial efforts in this capability.
- **Evidence:**
  - Partnership with the Arbor Day Foundation for tree distribution [Source](https://www.csrwire.com/press_releases/795501-delmarva-power-and-arbor-day-foundation-provide-free-trees-energy-efficient).

#### Capability 4: Resilience and Adaptation Actions
- **Summary:** Delmarva Power has taken initial resilience and adaptation actions focused on clean energy and climate goals, but specific resilience measures against climate impacts are not detailed.
- **Evidence:**
  - Major climate change commitment announcement [Source](https://www.delmarva.com/News/Pages/NewsReleases/DelmarvaPowerTakingMajorActiontoAdvanceCleanEnergy,CombatClimateChangeandBuildResilienceinMaryland.aspx).

#### Capability 5: Customer Engagement and Coordination
- **Summary:** Delmarva Power is actively engaging customers through energy-saving programs and assistance, showing initial steps towards customer engagement and coordination in the context of climate resilience.
- **Evidence:**
  - Energy assistance and energy-saving programs [Source](https://www.streetinsider.com/Accesswire/Delmarva+Power+Advocates+for+Continued+Energy+Assistance+Funding+and+Reminds+Customers+To+Act+Now+To+Secure+Support/22755948.html).

#### Capability 6: Attention to Equity
- **Summary:** Delmarva Power demonstrates an initial commitment to equity through grants for open space and resiliency projects, though direct evidence of active engagement in resilience planning with economically challenged communities is limited.
- **Evidence:**
  - Sustainable Communities Grant Program [Source](https://delawarebusinesstimes.com/news-briefs/delmarva-open-space-grants/).

In summary, Delmarva Power is taking initial steps across several capabilities defined in the Climate Resilience Maturity Model, with evidence of engagement in energy efficiency, advocacy for assistance programs, and environmental benefits. However, there is a need for more direct evidence of structured governance, detailed climate-aware planning, and specific resilience and adaptation actions directly addressing climate resilience.